# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/media/jay/DATA/learn/udacity-dend/udacity-data_modeling-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font>:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [6]:
session

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## Queries

### Question 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Question 1 expects Names of the artists, titles of the songs and lengths of the tracks based on sessionId and itemInSession.
As we are working with a NoSQL database, we need to think about the query first which will be used to fetch the data based on which we will create the Table required.

1) The expected output is: "Name of the artist, title of the song and length of the track"
2) Based on: "sessionId and itemInSession"

From the above two points we know the query to get the data will be a SELECT statement like (camelcase converted to snake_case):

```
SELECT artist_name, song_title, song_length FROM music_library_by_sessions WHERE session_id = value AND item_in_session = value
```

As we know the SELECT query, we can move to CREATE table query.

Before creating the table, we DROP the existing table if it exists, to ensure we start from a clean slate (i.e. to avoid any issue if the code is run twice).

We will add NOT EXIST to the CREATE statement to check if the table exists and only create the table if it does not exist. Now we need to select the columns that are going to be in the table and the PRIMARY KEY.

(Named: music_library_by_sessions as per Rubric requirements for tables names as alphanumeric; also as the details of the table is for songs and indexed by sessions, it sounds apt to name the table 'music_library_by_sessions')

Column Names: We need names of the artists, titles of the songs, and lengths of the tracks on query upon sessionId and itemInSession. Hence we will select artist_name, song_title , song_length, session_id and item_in_session as the names of the columns.

Primary Key: The PRIMARY key for the table should uniquely identify each row in the table. For us we need results based on sessionId and itemInSession; so we need these both as the primary key (Selecting one will throw filtering error on "SELECT * FROM music_library_by_sessions WHERE session_id = 338 and item_in_session = 4", as we have not set item_in_session in primary key; also filtering is not allowed for the project)

In [9]:
# CREATE

query = "DROP TABLE IF EXISTS music_library_by_sessions"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = """CREATE TABLE IF NOT EXISTS music_library_by_sessions
           (session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
# INSERT

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_by_sessions (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# SELECT

query = "SELECT artist_name, song_title, song_length FROM music_library_by_sessions WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
    for row in rows:
        print (row)
except Exception as e:
    print(e)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### Question 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Question 2 expects names of the artists, titles of the songs and users' full names based on userId and sessionId, and the data need to be sorted by itemInSession.

As the previous Question, we start by thinking about the query first.

1) The expected output is: "Name of the artist, title of the song and full name of the user"
2) Based on: "userId and sessionId"
3) Ordered by: "itemInSession"

From the above three points we know the query to get the data will be a SELECT statement like (camelcase converted to snake_case):

```
SELECT artist_name, song_title, item_in_session, user_name FROM music_library_by_sessions_and_users
WHERE session_id = value AND user_id = value ORDER BY item_in_session

```

**Note:** `ORDER BY` statement is not actually needed as we will cluster the table by `item_in_session`, I wrote it there just to clarify the query.

As we know the SELECT query, we can move to CREATE table query.

Before creating the table, we DROP the existing table if it exists, to ensure we start from a clean slate (i.e. to avoid any issue if the code is run twice).

We will add NOT EXIST to the CREATE statement to check if the table exists and only create the table if it does not exist. Now we need to select the columns that are going to be in the table and the PRIMARY KEY.

(Named: music_library_by_sessions_and_users as per Rubric requirements for tables names as alphanumeric; also as the details of the table is for songs and indexed by sessions and users, it sounds apt to name the table 'music_library_by_sessions_and_users')

Column Names: We need names of the artists, titles of the songs, and full names of the users on query upon sessionId and userId, ordered by itemInSession. Hence we will select artist_name , song_title, item_in_session, user_name, session_id and user_id as the names of the columns. For the user_name column, which is the user's full name, we concatenate both `firstName` and `lastName` from the csv file.

Primary Key: The PRIMARY key for the table should uniquely identify each row in the table. For us we need results based on sessionId and userId; so we need these both as the primary key

Clustering Column: the CLUSTERING COLUMN is the base of which the results will be ordered. Hence, we use itemInSession as the clustering column.

In [12]:
# CREATE

query = "DROP TABLE IF EXISTS music_library_by_sessions_and_users"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = """CREATE TABLE IF NOT EXISTS music_library_by_sessions_and_users
           (session_id int, user_id int, item_in_session int, artist_name text, song_title text, user_name text, PRIMARY KEY ((session_id, user_id), item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

    
# INSERT

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_by_sessions_and_users (session_id, user_id, item_in_session, artist_name, song_title, user_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], "{} {}".format(line[1], line[4])))

        
# SELECT

query = """SELECT artist_name, song_title, item_in_session, user_name FROM music_library_by_sessions_and_users
           WHERE session_id = 182 AND user_id = 10 ORDER BY item_in_session"""
try:
    rows = session.execute(query)
    for row in rows:
        print ("artist: {}, song: {} (item#: {}), user: {}".format(row.artist_name, row.song_title, row.item_in_session, row.user_name))
except Exception as e:
    print(e)           

artist: Down To The Bone, song: Keep On Keepin' On (item#: 0), user: Sylvie Cruz
artist: Three Drives, song: Greece 2000 (item#: 1), user: Sylvie Cruz
artist: Sebastien Tellier, song: Kilometer (item#: 2), user: Sylvie Cruz
artist: Lonnie Gordon, song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) (item#: 3), user: Sylvie Cruz


### Question 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Question 3 expects the users' full names based on a song they have listened to.

As the previous Questions, we start by thinking about the query first.

1) The expected output is: "Full name of the user"
2) Based on: "song title"

From the above two points we know the query to get the data will be a SELECT statement like (camelcase converted to snake_case):

```
SELECT user_name FROM users_by_songs_and_user_ids WHERE song_title = value

```

As we know the SELECT query, we can move to CREATE table query.

Before creating the table, we DROP the existing table if it exists, to ensure we start from a clean slate (i.e. to avoid any issue if the code is run twice).

We will add NOT EXIST to the CREATE statement to check if the table exists and only create the table if it does not exist. Now we need to select the columns that are going to be in the table and the PRIMARY KEY.

(Named: users_by_songs_and_user_ids as per Rubric requirements for tables names as alphanumeric; also as the details of the table is for users and indexed by song names and user ids, it sounds apt to name the table 'users_by_songs_and_user_ids')

Column Names: We need full names of the users on query upon a song title. We will also need to use user_id as one of the primary keys as values that are guaranteed to be unique (rather than user names). Hence we will select song_title, user_id, and user_name as the names of the columns. For the user_name column, which is the user's full name, we concatenate both `firstName` and `lastName` from the csv file.

Primary Key: The PRIMARY key for the table should uniquely identify each row in the table. For us we need results based on song and userId; so we need these both as the primary key.

In [13]:
# CREATE

query = "DROP TABLE IF EXISTS users_by_songs_and_user_ids"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = """CREATE TABLE IF NOT EXISTS users_by_songs_and_user_ids
           (song_title text, user_id int, user_name text, PRIMARY KEY (song_title, user_id))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

    
# INSERT

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_songs_and_user_ids (song_title, user_id, user_name)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), "{} {}".format(line[1], line[4])))

        
# SELECT

query = """SELECT user_name FROM users_by_songs_and_user_ids
           WHERE song_title='All Hands Against His Own'"""
try:
    rows = session.execute(query)
    for row in rows:
        print (row)
except Exception as e:
    print(e)           

Row(user_name='Jacqueline Lynch')
Row(user_name='Tegan Levine')
Row(user_name='Sara Johnson')


### Drop the tables before closing out the sessions

In [14]:
try:
    session.execute("DROP TABLE IF EXISTS music_library_by_sessions")
    session.execute("DROP TABLE IF EXISTS music_library_by_sessions_and_users")
    session.execute("DROP TABLE IF EXISTS users_by_songs")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()